# <u> Data Science Capstone project: battle of the neighborhoods</u>
### Connor Ruch

## Introduction/Business Problem

- Define a problem or an idea where i would need to leverage the Foursquare location data to solve or execute
- Lets say that a mechanic wants to open a new shop in the DC area. If he could know where there would be demand for a new shop and no too much other competition he could pick that area.
- I plan to use the foursquare location data to get locations of all mechanics in the Washington, DC area. I will leverage this data to predict where the prosctive business owner could open a new shop. 
- If i can find a location with crashes and few mechanics it might be a sign that there is demand for a new shop at that area. 
- my steakholders would be any mechanics looking to open a new shop.

## Data Used

- this report will use the foursquare API to get location data for all mechanics in the DC area
- I will also be using a dataset of vehicle crashses in the Washington, DC area. The data was downloaded from Kaggle at this url : "https://www.kaggle.com/gauravduttakiit/accidents-in-washington-dc"

Data explained:
- The foursquare data will list all the mechanics in the Washington, DC area including the latitude and longitude of the locations which will be important in mapping them. This data will be used by querying for matches of "Auto" and "mechanic" as the search terms. 
- The vehicle crash dataset includes crashes in the Washington, DC area. this dataset also includes latitude and longitude which is impertive for the mapping to the other dataset.

Examples below

In [2]:
# import libraries
import pandas as pd
import numpy as np
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import folium


# use geolocator to get coordinates
address = 'Washington, DC'

geolocator = Nominatim(user_agent='test_proj')
location = geolocator.geocode(address)
lat = location.latitude
long = location.longitude
print("Coordinates for Washington, DC: {}, {}".format(lat, long))

Coordinates for Washington, DC: 38.8949924, -77.0365581


In [1]:
# The code was removed by Watson Studio for sharing.

In [9]:
search_query = ['auto', 'mechanic'] #changed to blank
radius = 3000
print(search_query)

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}\
&query={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, lat, long,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

results = requests.get(url).json()

['auto', 'mechanic']


#### - Below is an example of the dataframe that would result when getting this data from foursquare
####  - Below the first dataframe is an example of the data from the vehicle accident data

In [10]:
venues = results['response']['venues']
df = json_normalize(venues)
df.head()

c:\users\jiminey\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.address,location.crossStreet,venuePage.id,location.neighborhood
0,4c0e872cd64c0f474e57285d,Auto Alliance,[],v-1624585645,False,38.902200,-77.033240,"[{'label': 'display', 'lat': 38.9022, 'lng': -...",852,20005,US,Washington,D.C.,United States,"[Washington, D.C. 20005, United States]",NaN,NaN,NaN,NaN
1,51e7ea1f498e25214b5aa4e5,Auto Alliance,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1624585645,False,38.900070,-77.021512,"[{'label': 'display', 'lat': 38.90007, 'lng': ...",1420,NaN,US,Washington,D.C.,United States,"[803 7th Street, NW, Washington, D.C., United ...","803 7th Street, NW",NaN,NaN,NaN
2,4dcc714c887714552bd6ce29,Auto Ching Ching Mercedes,"[{'id': '4bf58dd8d48988d1f9931735', 'name': 'R...",v-1624585645,False,38.905593,-77.043451,"[{'label': 'display', 'lat': 38.90559342916098...",1322,20001,US,Washington,D.C.,United States,"[Washington, D.C. 20001, United States]",NaN,NaN,NaN,NaN
3,4c44b3f9f799e21ef633aea2,Automated Graphic Imaging,[],v-1624585645,False,38.905868,-77.032239,"[{'label': 'display', 'lat': 38.905868, 'lng':...",1267,20005,US,Washington,D.C.,United States,"[1090 Vermont Ave NW, Lowr LEVEL (Vermont and ...","1090 Vermont Ave NW, Lowr LEVEL",Vermont and L,NaN,NaN
4,4cc89943d54fa1cdb28d3229,Exotic Auto Detail,"[{'id': '4f04ae1f2fb6e1c99f3db0ba', 'name': 'C...",v-1624585645,False,38.904973,-77.048837,"[{'label': 'display', 'lat': 38.90497265713505...",1538,20037,US,Washington,D.C.,United States,"[1150 22nd St NW, Washington, D.C. 20037, Unit...",1150 22nd St NW,NaN,57955077,NaN


In [12]:
dc_crash_df = pd.read_csv("Washington_accidents.csv")

latitudes = []
for lat, long, label in zip(dc_crash_df['Start_Lat'], dc_crash_df['Start_Lng'], dc_crash_df['ID']):
    temp_dic = {}
    if lat > 38.93 or lat < 38.85:
        pass 
    else:
        if long > -77.01 or long < -77.06:
            pass
        else:
            temp_dic['ID'] = label
            temp_dic['lat'] = lat
            temp_dic['long'] = long
    if temp_dic:
        latitudes.append(temp_dic)
    
#print(latitudes)
dc_df = pd.DataFrame(latitudes)
print(dc_df.shape)
dc_df.sort_values(by='lat', ascending=True).head()

(1426, 3)


,ID,lat,long
805,A-2575025,38.872988,-77.042354
1189,A-3309587,38.873060,-77.042273
991,A-2884633,38.873160,-77.042170
975,A-2871809,38.873160,-77.042170
974,A-2871799,38.873160,-77.042170
